In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle

from modules.logging.format_utils import format_measures
from modules.collecting.results_collector import DataFrameCollector
from modules.logging.logger import DefaultLogger
from modules.algorithms.base.OSLPP import Params
from modules.selection.uncertanties import SelectRejectMode
from modules.algorithms.nn.OSLPP_NN_UTILS import train
from modules.types.selection_types import SelectRejectMode

In [ ]:
with open('experiments/configs/small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
# sources = ['art', 'clipart', 'product', 'real_world']
# targets = ['art', 'clipart', 'product', 'real_world']

In [ ]:
# sources = ['amazon', 'dslr', 'webcam']
# targets = ['amazon', 'dslr', 'webcam']

In [ ]:
# sources = ['painting_train', 'real_train', 'sketch_train']
# targets = ['painting_train', 'real_train', 'sketch_train']

In [ ]:
# sources = ['train'] 
# targets = ['validation']

In [ ]:
sources = ['painting_train', 'real_train', 'sketch_train']
targets = ['painting_test', 'real_test', 'sketch_test']

In [ ]:
results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': [], 'mode': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
# toptop = [SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY]
logger = DefaultLogger()
for source in sources:
    for target in targets:
        if source.split('_')[0] == target.split('_')[0]:
            continue
        for epochs in [10]:
            for lr in [1e-3]:
                for n_r in [0.15]:
                    for select_reject_mode in [SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY, SelectRejectMode.MARGIN]:
                        for seed in range(5):
                            params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                            dataset='DomainNet_DCC', source=source, target=target,
                                            num_common=150, num_src_priv=50, num_tgt_priv=145)
                            metrics = train(params, lr, epochs, 1, select_reject_mode, seed, range(200), None, logger, osda=False)
                            results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs, 'mode': select_reject_mode})
results.get_result().to_csv('all_results/results/dcc__UNDA_ALL_MODES.csv', header=True, index=False)

In [ ]:
# results.get_result().to_csv('all_results/results/dcc__CONF_VALID__nn_raw.csv', header=True, index=False)

In [ ]:
sources = ['art', 'clipart', 'product', 'real_world']
targets = ['art', 'clipart', 'product', 'real_world']

In [ ]:
results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
# toptop = [SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY]
logger = DefaultLogger()
for source in sources:
    for target in targets:
        for epochs in [10]:
            for lr in [1e-3]:
                for n_r in [0.15]:
                    for seed in range(1):
                        params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                        dataset='OfficeHome', source=source, target=target,
                                        num_common=10, num_src_priv=5, num_tgt_priv=50)
                        metrics = train(params, lr, epochs, 1, select_reject_mode, seed, range(10), None, logger, osda=False)
                        results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs})
# results.get_result().to_csv('all_results/results/aaa.csv', header=True, index=False)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('all_results/results/office_31_oslpp_nn_raw.csv')

In [ ]:
df['h_score'] = df['desc'].apply(lambda x: float(x.split()[-1].split('=')[-1]))

In [ ]:
df

In [ ]:
df.groupby(['source', 'target', 'lr', 'n_r'])['h_score'].mean().to_frame(
    name='h_score').reset_index().sort_values(by='h_score', ascending=False).drop_duplicates(['source', 'target'])

In [ ]:
sources = ['amazon', 'dslr', 'webcam']
targets = ['amazon', 'dslr', 'webcam']

In [ ]:
results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
logger = DefaultLogger()
for source in sources:
    for target in targets:
        if source.split('_')[0] == target.split('_')[0]:
            continue
        for epochs in [10]:
            for lr in [1e-3]:
                for n_r in [0.15]:
                    for seed in range(9):
                        params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                        dataset='Office31', source=source, target=target,
                                        num_common=10, num_src_priv=10, num_tgt_priv=11)
                        metrics = train(params, lr, epochs, 1, select_reject_mode, seed, range(200), None, logger, osda=False)
                        results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs})
results.get_result().to_csv('all_results/results/office31_conf.csv', header=True, index=False)